## Выбор фреймворка




Используйте ноутбук, чтобы по шагам запускать загрузку моков, выбор фреймворка и кодогенерацию.

In [1]:
from mas_automl.code_agent.load_mocks import load_mock_inputs

DATA_ANALYZE, METADATA, REGISTRY, FINAL_DATA = load_mock_inputs()
print(f"Ключи data: {list[str](DATA_ANALYZE.keys())}")
print(f"Ключи metadata: {list(METADATA.keys())}")
print(f"Фреймворки: {list(REGISTRY.keys())}")


Ключи data: ['summary', 'priority', 'steps', 'example_pipeline_snippet', 'frameworks_recommended', 'rationale', 'estimated_complexity', 'confidence']
Ключи metadata: ['dataset_id', 'name', 'version', 'version_label', 'description', 'citation', 'paper_url', 'paper_reference', 'creator', 'contributor', 'collection_date', 'upload_date', 'language', 'licence', 'url', 'original_data_url', 'minio_url', 'format', 'file_id', 'default_target_attribute', 'ignore_attribute', 'row_id_attribute', 'num_rows', 'num_features', 'num_classes', 'num_missing_values', 'quality', 'dataset_type', 'tags', 'status', 'visibility', 'extra_info', 'local_path']
Фреймворки: ['AutoGluon', 'H2O AutoML', 'LightAutoML']


In [17]:
from mas_automl.code_agent.load_mocks import load_mock_inputs
from IPython.display import display, Markdown

DATA_ANALYZE, METADATA, REGISTRY, FINAL_DATA = load_mock_inputs()

# Pretty display in notebook
display(Markdown(f"""
### Loaded Mock Inputs

#### Data Analyze Keys
{', '.join(DATA_ANALYZE.keys())}

#### Metadata Keys
{', '.join(METADATA.keys())}

#### Registry Frameworks
{', '.join(REGISTRY.keys())}

{', '.join(FINAL_DATA.keys())}
"""))





### Loaded Mock Inputs

#### Data Analyze Keys
summary, priority, steps, example_pipeline_snippet, frameworks_recommended, rationale, estimated_complexity, confidence

#### Metadata Keys
dataset_id, name, version, version_label, description, citation, paper_url, paper_reference, creator, contributor, collection_date, upload_date, language, licence, url, original_data_url, minio_url, format, file_id, default_target_attribute, ignore_attribute, row_id_attribute, num_rows, num_features, num_classes, num_missing_values, quality, dataset_type, tags, status, visibility, extra_info, local_path

#### Registry Frameworks
AutoGluon, H2O AutoML, LightAutoML

manifest, validation_report, metafeatures, preprocessing_recipe, code_agent_recommendation, run_metadata


In [18]:
REGISTRY

{'AutoGluon': "AutoGluon — AutoML-фреймворк AWS для табличных, текстовых, визуальных и мультимодальных данных. Использует ансамблирование (stacking, bagging, blending) и автоматический подбор гиперпараметров. Архитектура модульная: AutoGluon-Tabular, AutoGluon-Multimodal, AutoGluon-TimeSeries. Поддерживает GPU, распределённое обучение и масштабирование на больших наборах данных. \n\nГде применять: универсальные задачи классификации и регрессии на табличных данных; задачи анализа изображений (classification, object detection) и текстов (sentiment analysis, NER) через AutoMM; смешанные типы данных (таблица + текст/изображение); Kaggle/продакшн-сценарии, где нужно получить максимум качества без ручной настройки. \n\nТехнические возможности: автоматическое кодирование категориальных признаков; генерация новых признаков; встроенный HPO (Bayesian Optimization + Random Search); сохранение и загрузка моделей; интеграция с PyTorch и MXNet; поддержка обучения на GPU. \n\nОграничения: плохо подхо

In [19]:
FINAL_DATA.get('local_path')


In [22]:
FINAL_DATA["manifest"]["local_path"]


'C:\\Users\\User1\\Desktop\\ITMO_bootcamp\\data\\datasets\\openml_31_credit-g.csv'

In [23]:
# DATA_ANALYZE, METADATA, REGISTRY
import pandas as pd

PATH_TO_CSV = FINAL_DATA["manifest"]["local_path"]
dataset_df = pd.read_csv(PATH_TO_CSV)

dataset_df.info(), dataset_df.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   checking_status         1000 non-null   object 
 1   duration                1000 non-null   int64  
 2   credit_history          1000 non-null   object 
 3   purpose                 1000 non-null   object 
 4   credit_amount           1000 non-null   float64
 5   savings_status          1000 non-null   object 
 6   employment              1000 non-null   object 
 7   installment_commitment  1000 non-null   int64  
 8   personal_status         1000 non-null   object 
 9   other_parties           1000 non-null   object 
 10  residence_since         1000 non-null   int64  
 11  property_magnitude      1000 non-null   object 
 12  age                     1000 non-null   int64  
 13  other_payment_plans     1000 non-null   object 
 14  housing                 1000 non-null   o

(None,
   checking_status  duration                  credit_history   purpose  \
 0              <0         6  critical/other existing credit  radio/tv   
 1        0<=X<200        48                   existing paid  radio/tv   
 
    credit_amount    savings_status employment  installment_commitment  \
 0         1169.0  no known savings        >=7                       4   
 1         5951.0              <100     1<=X<4                       2   
 
       personal_status other_parties  ...  property_magnitude age  \
 0         male single          none  ...         real estate  67   
 1  female div/dep/mar          none  ...         real estate  22   
 
    other_payment_plans housing existing_credits      job num_dependents  \
 0                 none     own                2  skilled              1   
 1                 none     own                1  skilled              1   
 
    own_telephone foreign_worker class  
 0            yes            yes  good  
 1           none       

In [5]:
# --- Инициализация клиента ---
from mas_automl.code_agent.openai_wraper import LLMClient, LLMConfig

client = LLMClient(LLMConfig())

# --- Вызов выбора фреймворка ---


🔑 OPENAI_API_KEY = sk-or-v
🌐 BASE_URL = https://openrouter.ai/api/v1
KWARGS - {'model': 'qwen/qwen3-coder-30b-a3b-instruct', 'temperature': 0.0, 'openai_api_key': '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', 'openai_api_base': 'https://openrouter.ai/api/v1'}


C:\Users\User1\Desktop\ITMO_bootcamp\src\mas_automl\code_agent\openai_wraper.py:50: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the `langchain-openai package and should be used instead. To use it run `pip install -U `langchain-openai` and import as `from `langchain_openai import ChatOpenAI``.
  self._client = ChatOpenAI(**client_kwargs)


✅ ChatOpenAI клиент инициализирован для OpenRouter


In [6]:
client.chat('loh')

'Hello! It looks like you might have accidentally sent "loh" - is there something I can help you with? I\'m here to assist with questions, conversations, or any other support you might need. Feel free to let me know how I can help!'

## Выбор фреймворка

## Итеративная генерация и тестирование кода

Ниже показан процесс итеративной генерации кода с тестированием через execnet_gateway


In [ ]:
# Итеративная генерация и тестирование кода
from mas_automl.code_agent.base_pipeline import generate_code, evaluate_code
from mas_automl.code_agent.execnet_gateway import PythonSandboxClient
from pathlib import Path
import pandas as pd

CSV_PATH = FINAL_DATA["manifest"]["local_path"]
OUTPUT_DIR = str(Path(CSV_PATH).parent / "predictions")
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

framework = "scikit-learn"
max_iterations = 3
feedback = ""

print(f"CSV путь: {CSV_PATH}")
print(f"Директория для предиктов: {OUTPUT_DIR}\n")

for iteration in range(1, max_iterations + 1):
    print(f"{'='*60}")
    print(f"Итерация {iteration}/{max_iterations}")
    print(f"{'='*60}\n")
    
    # Генерируем код
    print("Генерация кода...")
    code = generate_code(framework, client, iteration, feedback, FINAL_DATA)
    print(f"Сгенерированный код:\n{code}\n")
    
    # Тестируем код
    print("Тестирование кода...")
    tests_passed, test_feedback, predict_path = evaluate_code(
        code, framework, CSV_PATH, OUTPUT_DIR, iteration, FINAL_DATA
    )
    
    print(f"Тесты пройдены: {tests_passed}")
    print(f"Обратная связь: {test_feedback}")
    if predict_path:
        print(f"Путь к предиктам: {predict_path}")
        # Загружаем и показываем предикты
        try:
            pred_df = pd.read_csv(predict_path)
            print(f"\nПервые 5 строк предиктов:")
            print(pred_df.head())
        except Exception as e:
            print(f"Ошибка загрузки предиктов: {e}")
    
    print("\n")
    
    if tests_passed:
        print("✅ Все тесты пройдены! Останавливаем итерации.")
        break
    else:
        feedback = test_feedback
        print("❌ Тесты не пройдены. Переходим к следующей итерации с учетом обратной связи.\n")

print(f"\n{'='*60}")
print("Итоговый результат:")
print(f"Итераций выполнено: {iteration}")
print(f"Тесты пройдены: {tests_passed}")
print(f"Путь к предиктам: {predict_path}")


## Просмотр сохраненных предиктов

Если предикты были успешно сохранены, можно загрузить и проанализировать их


In [ ]:
# Просмотр сохраненных предиктов
from pathlib import Path
import pandas as pd

if predict_path and Path(predict_path).exists():
    pred_df = pd.read_csv(predict_path)
    print(f"Загружено предиктов: {len(pred_df)}")
    print(f"\nПервые 10 строк:")
    print(pred_df.head(10))
    print(f"\nСтатистика:")
    print(pred_df.describe())
    
    # Простая метрика точности
    if 'true_label' in pred_df.columns and 'prediction' in pred_df.columns:
        from sklearn.metrics import accuracy_score
        accuracy = accuracy_score(pred_df['true_label'], pred_df['prediction'])
        print(f"\nТочность (accuracy): {accuracy:.4f}")
else:
    print("Предикты не найдены. Запустите предыдущую ячейку для генерации.")


In [ ]:
from IPython.display import display, Markdown

# Your existing call (unchanged)
from mas_automl.code_agent.base_pipeline import choose_framework

framework, reason, prompt = choose_framework(
    DATA_ANALYZE, METADATA, REGISTRY, FINAL_DATA, client)

# Pretty display in notebook
display(Markdown(f"""
### Selected Framework
**Framework:** {framework}

### Reasoning
{reason}

### Generated Prompt
{prompt}
"""))



### Selected Framework
**Framework:** H2O AutoML

### Reasoning
H2O AutoML является наиболее стабильным и универсальным выбором для табличных данных данного размера (1000 строк, 21 признаков) и типа задачи (бинарная классификация). Оно предлагает надежный стекинг моделей (GBM, XGBoost, GLM, DRF, DeepLearning), автоматическую кросс-валидацию, раннюю остановку и поддержку production-сценариев. Несмотря на отсутствие поддержки мультимодальности и GPU для всех моделей, оно хорошо подходит для задач классификации с высокой воспроизводимостью и стабильностью, что особенно важно в корпоративных и промышленных условиях. В отличие от AutoGluon, которое требует больше ресурсов и может быть менее предсказуемо на небольших наборах данных, H2O AutoML обеспечивает более предсказуемое поведение и более широкую совместимость с существующими инфраструктурными решениями.

### Generated Prompt
input_variables=['analysis_json', 'frameworks_list', 'metadata_json'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Ты — эксперт по AutoML и ML-инженер. Твоя задача — выбрать наиболее подходящий AutoML-фреймворк для табличных данных. Оцени качество данных, размер, тип задачи, ограничения и предложи лучший вариант из списка.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['analysis_json', 'frameworks_list', 'metadata_json'], input_types={}, partial_variables={}, template='Вот анализ датасета и метаданные:\n\n### 📊 Data Analysis\n{analysis_json}\n\n### 🧾 Metadata\n{metadata_json}\n\n### ⚙️ Доступные AutoML фреймворки\n{frameworks_list}\n\nПоясни свой выбор кратко, но содержательно. Если несколько подходят, выбери наиболее универсальный и стабильный вариант. Ответ верни в формате JSON:\n{{"framework": "...", "reason": "..."}}'), additional_kwargs={})]


In [7]:
print(type(prompt))

<class 'langchain_core.prompts.chat.ChatPromptTemplate'>


# Кодогенерация

In [8]:
from mas_automl.code_agent.base_pipeline import generate_code, evaluate_code

code_sample = generate_code(framework, client, iteration=1, feedback="")
print(code_sample)
print(evaluate_code(code_sample, framework))


import h2o
from h2o.automl import H2OAutoML

def train_model(train_df, test_df, label):
    h2o.init()
    train = h2o.H2OFrame(train_df)
    test = h2o.H2OFrame(test_df)
    train[label] = train[label].asfactor()
    test[label] = test[label].asfactor()
    
    aml = H2OAutoML(max_models=10, seed=1)
    aml.train(x=train.columns, y=label, training_frame=train)
    
    return aml.leader
(True, 'Проверки пройдены.')


In [11]:
from mas_automl.code_agent.base_pipeline import run_pipeline

result = run_pipeline(max_iterations=2, llm=client)
result


ValueError: too many values to unpack (expected 3)